In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("train_dataset.csv")
test = pd.read_csv("test_dataset.csv")

In [20]:
train.columns

Index(['Product', 'Product_Brand', 'Item_Category', 'Subcategory_1',
       'Subcategory_2', 'Item_Rating', 'Date', 'Selling_Price'],
      dtype='object')

In [19]:
train.Item_Category.value_counts()

Item_Category
clothing                                            605
jewellery                                           359
automotive                                          165
footwear                                            131
computers                                            92
                                                   ... 
tadd men s women s ankle length socks                 1
olvin aviator sunglasses                              1
sonaxo men running shoes                              1
favourite bikerz 3514 rad air filter ionic air f      1
say uv sterilizer solid filter cartridge 0 pac        1
Name: count, Length: 67, dtype: int64

You are given the following dataset. As you can probably already infer, it is from an e-commerce platform. Your task is to try and come up with a solid ML model that can predict the price of the good given those 7 columns. Please use the already given train test split.

### EDA and preparing final dataset

Please don't spend a lot of time with the EDA, just a few graphs to know what to do with the variables and move on. Please explain your reasoning for every step whenever you can.

Hint: How is the selling price distributed?

### Fitting model

Please fit the best model you can to your adapted dataset. Fit a model of your choice.

Please explain your reasoning for every step whenever you can.